In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from dataloader import SplicingKmerDataset

In [3]:
from kipoi.data import DataLoader
from kipoi.data_utils import numpy_collate

In [4]:
from model import HALModel

In [5]:
fasta = "/s/genomes/human/hg19/ensembl_GRCh37.p13_release75/Homo_sapiens.GRCh37.75.dna.primary_assembly.fa"
GFF = "/s/genomes/human/hg19/MISO/A5SS.hg19.nochr.selected.gff3"

In [6]:
data = SplicingKmerDataset(GFF, 
                           fasta,
                           MISO_AS=True) #get_A5SS_dt, otherwise return a dicionary with inputs, targets...

In [7]:
test = data[2]

test

{'inputs': array([ 'CCGCGCTACTTCGGCCCCAACTTGCTCAACACGGTGAAGCAGAAGCTCTTCACCGAGGTGGAGGGGACCTGCACAGGGAAGTGAGTGTCGAGCTCACCGCACCGCCAGATCGTTCGATGCGCACACGGGGCGCTATACCTGCAACCCCTCCCCAACTCCT',
        'GAGATGGGTCATTCATTGACTCCTCAAACATTTTAAAAACCCTTGGTGTGTGCCAGGACCCGTGTTGGGCTCAGAACAAGGTAGAATGGGAACAAGATAGAATGAGGACAGCCCCTGTGGAGCTCACAGTGTAATCATTGTTCCTGTGTCTGCTCTCCCT'],
       dtype='<U160'),
 'metadata': {'chrom': '11',
  'geneName': 'chr11:62529267:62529376|62529656:+@chr11:62530339:62530498:+',
  'start': 62529267,
  'stop': 62530498,
  'strand': '+'}}

In [8]:
seqs = test['inputs']

In [9]:
seqs[0][-80:]

'GTGAGTGTCGAGCTCACCGCACCGCCAGATCGTTCGATGCGCACACGGGGCGCTATACCTGCAACCCCTCCCCAACTCCT'

In [10]:
seqs[0][80:]

'GTGAGTGTCGAGCTCACCGCACCGCCAGATCGTTCGATGCGCACACGGGGCGCTATACCTGCAACCCCTCCCCAACTCCT'

## Prediction

In [11]:
data_generator = DataLoader(data, batch_size=3, collate_fn=numpy_collate, shuffle=True)
data_generator = iter(data_generator)

In [12]:
data = next(data_generator)

In [13]:
data['inputs']

array([[ 'TGCAGAGCCGAAGAGAAGATGAAACACGAACCCAGAGGAGCAGGATGACAGACAGAGTCCCTTGTTCCAAATGTTTCCAGGTTCACTGCCAGATTTCTGTTCTGAATTGTGTAAGACAAACTGCTATCCTGATTTGCTCGTGTTTGTTAAGCTAGTTCAA',
        'CCCAGAGGAGCAGGATGACAGACAGAGTCCCTTGTTCCAAATGTTTCCAGGTTCACTGCCAGATTTCTGTTCTGAATTGTGTAAGACAAACTGCTATCCTGATTTGCTCGTGTTTGTTAAGCTAGTTCAAGTTGGGTTTCCATCACATGCAACCAAGTGA'],
       [ 'CCTTCCTCCTTCCCATTTCGGCGGACCTGCATCCAGCTAGAGACTAGCAGATTGAATCGATATTCATTAAGTTAGGAATGGTTGGTGGTCCTAAGAGCCTCAGCAACCGAGAGGAGTCTTGTTATTTAAAAAACAACGATACTGGGCGTGGTCGTGGGCC',
        'AAAACAACGATACTGGGCGTGGTCGTGGGCCTGTAGTCCCAGCTGCTCAAGCTAGTGGATCTCTTGAGTCCAAGGCCAATGTGAGCAACATAGCAAGACCCAGTCTCAAAAAATAAATACATAATAATAATAAAAAACCCTCGTTCACCTAAACAGATCT'],
       [ 'AAAAGCAGGACCCACAAGAATGTGAAAGCCTCAGTCTTGCAGCCTCTCTGATGTCTTGTGGGTAATACAGTGGAGATGTGGTAGGAAAAGCTCACAGCCTTCCTCCATAGGCTTCTGGTCTCACTTTCTGCACTGCTGGCCTCTTACAACAGGTTTTCGC',
        'GCCTCTTACAACAGGTTTTCGCAATCAGGCTGAGCTTAAGATAAACAGATTGCCTCCTCCCTGGAAAATCTAGGCTCTTGGTACGTCCTTGACTGAAGATGATCCACATCTGCTTGGGAAGATCCG

In [14]:
HAL = HALModel()

In [15]:
HAL.predict_on_batch(next(data_generator))

array([ 0.98349686,  0.19322312,  0.01905562])

# With normal ensembl gtf annotation

In [16]:
fasta = "/s/genomes/human/hg19/ensembl_GRCh37.p13_release75/Homo_sapiens.GRCh37.75.dna.primary_assembly.fa"
GFF = "/s/genomes/human/hg19/ensembl_GRCh37.p13_release75/Homo_sapiens.GRCh37.75.chr22.gtf"

In [17]:
data = SplicingKmerDataset(GFF, 
                           fasta,
                           MISO_AS=False)

In [18]:
data[0]

{'inputs': 'TAGCCATCAGGCCTGGGGATGGACCTCCCTGGAGCTGGGCATTCAAGAGGATGAGTGCAGCAGAACGCGTATCCAGGGAGGTCTGGTCTGCTGATTTTTCTAGGCATAAAGCAAATTTATTTTTTAAATTAAAATAAAAATGCATGCACTTTCAAATGTA',
 'metadata': {'biotype': 'lincRNA',
  'chrom': '22',
  'geneID': 'ENSG00000233866',
  'order': 0,
  'start': 16062237,
  'stop': 16062396,
  'strand': '+',
  'transcriptID': 'ENST00000424770'}}

In [19]:
data_generator = DataLoader(data, batch_size=5, collate_fn=numpy_collate, shuffle=False)
data_generator = iter(data_generator)

In [20]:
data = next(data_generator)

In [21]:
data

{'inputs': array([ 'TAGCCATCAGGCCTGGGGATGGACCTCCCTGGAGCTGGGCATTCAAGAGGATGAGTGCAGCAGAACGCGTATCCAGGGAGGTCTGGTCTGCTGATTTTTCTAGGCATAAAGCAAATTTATTTTTTAAATTAAAATAAAAATGCATGCACTTTCAAATGTA',
        'GAATGTACAGACTAGCACAGAAGTTGGGCTAATTAAACAAGTATTGCTGAAAATGAGTGCTGTAGATGACATGAGAGCAGGTATGGGGTTGATTGTTAGGGAAGTATAACTTAAAAGTTTATAAAGTTTCACATACTTCTCTTTATATTCTATAGGTAAT',
        'TGGCAACTGTTTTATAGTCACATCATCGAAGTCCAAAGGAAAATGTTTTCAGCATTGTGTGAAATATGATTTTCAACCACGTAAGGTAGGTAAAAGTAAATATTTTTATAACTCACTTGTTATGACAGAATTCTTAACTATCCTTTTATGACCTCTGTAG',
        'ATACTCAACTCTGCCTGGAATCATCACCAAAAGAGGATGCATCAATTTTTGTGCATTCCCAACATGCTCTAATGCTTCCGGTGGGTGAATCATGGCTTTGTTTTCATGTTCTTGTCAGAATTTAACAATATTTTTATTTTATATATCAAACATTGCTCAT',
        'GCAGCGGCGTGGTCAGAGCGAGCTTCGGAGAAGCAGTGGTGGGTTCCATGTGATGGTGGAGTAGGAGGCAGGTCTCCGCGGTAAGTGGCGGGGGCGTGGACCCCACCGGGAACCCTCCCGGCTCCTTCCCTGCCTCTCCCTGTTTTCGTGCTTTCACTTC'],
       dtype='<U160'),
 'metadata': {'biotype': array(['lincRNA', 'pseudogene', 'pseudogene', 'pseudogene',
         'processed_

In [22]:
data['inputs']

array([ 'TAGCCATCAGGCCTGGGGATGGACCTCCCTGGAGCTGGGCATTCAAGAGGATGAGTGCAGCAGAACGCGTATCCAGGGAGGTCTGGTCTGCTGATTTTTCTAGGCATAAAGCAAATTTATTTTTTAAATTAAAATAAAAATGCATGCACTTTCAAATGTA',
       'GAATGTACAGACTAGCACAGAAGTTGGGCTAATTAAACAAGTATTGCTGAAAATGAGTGCTGTAGATGACATGAGAGCAGGTATGGGGTTGATTGTTAGGGAAGTATAACTTAAAAGTTTATAAAGTTTCACATACTTCTCTTTATATTCTATAGGTAAT',
       'TGGCAACTGTTTTATAGTCACATCATCGAAGTCCAAAGGAAAATGTTTTCAGCATTGTGTGAAATATGATTTTCAACCACGTAAGGTAGGTAAAAGTAAATATTTTTATAACTCACTTGTTATGACAGAATTCTTAACTATCCTTTTATGACCTCTGTAG',
       'ATACTCAACTCTGCCTGGAATCATCACCAAAAGAGGATGCATCAATTTTTGTGCATTCCCAACATGCTCTAATGCTTCCGGTGGGTGAATCATGGCTTTGTTTTCATGTTCTTGTCAGAATTTAACAATATTTTTATTTTATATATCAAACATTGCTCAT',
       'GCAGCGGCGTGGTCAGAGCGAGCTTCGGAGAAGCAGTGGTGGGTTCCATGTGATGGTGGAGTAGGAGGCAGGTCTCCGCGGTAAGTGGCGGGGGCGTGGACCCCACCGGGAACCCTCCCGGCTCCTTCCCTGCCTCTCCCTGTTTTCGTGCTTTCACTTC'],
      dtype='<U160')

In [23]:
HAL = HALModel()

In [24]:
HAL.predict_on_batch(next(data_generator))

array([ 16.58693109,  11.91657679,  13.674915  ,  15.40467572,  12.35613842])